<a href="https://colab.research.google.com/github/jain777/Technical-Indicators-FA-Project/blob/main/Future_Prediction_EMA_5_10_20_%2B_RSI_%2B_MFI_%2B_STOCHRSI_%2B_OBV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [97]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [98]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web
import pandas_ta as ta
import matplotlib.pyplot as plt
from datetime import date
plt.style.use('fivethirtyeight')
yf.pdr_override()

In [163]:
stocksymbols = ['SBIN.NS']
startdate = date(2015,1,1)
end_date = date(2015,3,31)
def getMyPortfolio(stocks = stocksymbols ,start = startdate , end = end_date):
    data = web.get_data_yahoo(stocks , data_source='yahoo' , start = start ,end= end )
    return data

In [164]:
df = getMyPortfolio(stocksymbols)


[*********************100%***********************]  1 of 1 completed


In [165]:
CustomStrategy = ta.Strategy(
    name="Defining Various TI to be used",
    description="MFI,RSI & EMA 30,50,100",
    ta=[
        {"kind": "mfi",}, #volume
        {"kind" : "obv"}, #volume
        {"kind": "rsi"}, #momentum
        {"kind":"stochrsi"},#momentum
        {"kind": "ema", "length": 5}, #trend
        {"kind": "ema", "length": 10}, #trend
        {"kind": "ema", "length": 20}, #trend
    ]
)
df.ta.strategy(CustomStrategy)

In [166]:
df.rename(columns = {'EMA_5':'Short', 'EMA_10':'Middle', 'EMA_20':'Long'}, inplace = True)
df['OBV_EMA'] = df['OBV'].ewm(span=14).mean()

In [167]:
def buy_sell_function(data):
  buysig = []
  buyprice = []
  flag_long = False
  flag_short = False
  for i in range(0,len(data)):
    if data['Middle'][i] < data['Long'][i] and data['Middle'][i] > data['Short'][i] and flag_long == False and flag_short == False:
        buysig.append(1)
        buyprice.append(data['Close'][i])
        flag_short = True
    elif data['Middle'][i] > data['Long'][i] and data['Middle'][i] < data['Short'][i] and flag_short == False and flag_long == False:
        buysig.append(1)
        buyprice.append(data['Close'][i])
        flag_long = True
    elif flag_short == True and data['Short'][i] > data['Middle'][i]:
        buyprice.append(data['Close'][i])
        buysig.append(-1)
        flag_short = False
    elif flag_long == True and data['Short'][i] < data['Middle'][i]:
        buyprice.append(data['Close'][i])
        buysig.append(-1)
        flag_long = False
    else:
        buysig.append(0)
        buyprice.append(data['Close'][i])
  return (buysig, buyprice)

df['buysignal'] = buy_sell_function(df)[0]
df['buyprice'] =buy_sell_function(df)[1]

In [168]:
obv=df['OBV']
obvema = df['OBV_EMA']
fl=0
obvsig=[]
for i in range(len(obv)):
  if obv[i]<obvema[i] and fl==1:
    obvsig.append(-1)
    fl=0
  elif obv[i]>obvema[i] and fl==0:
    obvsig.append(1)
    fl=1
  else:
    obvsig.append(0)

In [169]:
stoch=df['STOCHRSIk_14_14_3_3']
stochsig=[]
for i in range(len(stoch)):
  if stoch[i]>80:
    stochsig.append(-1)
  elif stoch[i]<20:
    stochsig.append(1)
  else:
    stochsig.append(0)

In [170]:
mfi=df['MFI_14']
rsi=df['RSI_14']
mfisig=[]
for i in range(len(mfi)):
  if mfi[i]>80:
    mfisig.append(-1)
  elif mfi[i]<20:
    mfisig.append(1)
  else:
    mfisig.append(0)

In [171]:
rsi=df['RSI_14']
rsisig=[]
for i in range(len(rsi)):
  if rsi[i]>70:
    rsisig.append(-1)
  elif rsi[i]<30:
    rsisig.append(1)
  else:
    rsisig.append(0)

In [172]:
bs=df['buysignal']
ss=df['buyprice']

In [173]:
def calcProfit(w1,w2,w3,w4,w5):
  tp=0
  fl=0
  bp=0
  for i in range(len(bs)):
    val= mfisig[i]*w1 + obvsig[i]*w2 + rsisig[i]*w3 + stochsig[i]*w4 + bs[i]*w5 
    if val>=20 and fl==0:
      tp-=ss[i]
      fl=1
      bp=ss[i]
    elif val<=-20 and fl==1:
      tp+=ss[i]
      fl=0

  if(fl==1):
    tp+=bp

  return tp

arr=[0,10,20,30,40,50,60,70,80]
ans=0
wt=[]
for i1 in range(len(arr)):
  for i2 in range(len(arr)):
    for i3 in range(len(arr)):
      for i4 in range(len(arr)):
        for i5 in range(len(arr)):
          funcval=calcProfit(arr[i1],arr[i2],arr[i3],arr[i4],arr[i5])
          ans=max(funcval,ans)
          if(funcval==ans):
            wt.clear()
            wt.append(arr[i1])
            wt.append(arr[i2])
            wt.append(arr[i3])
            wt.append(arr[i4])
            wt.append(arr[i5])

fin=(ans/ss[0])*100
marketprof=((ss[len(ss)-1]-ss[0])/ss[0])*100
print(fin)
print(marketprof)
print(wt)


11.894925840341362
-14.681530606215167
[80, 80, 80, 80, 10]


In [ ]:
stocksymbols = ['SBIN.NS']
startdate = date(2015,4,1)
end_date = date(2015,6,30)
def getMyPortfolioNew(stocks = stocksymbols ,start = startdate , end = end_date):
    data = web.get_data_yahoo(stocks , data_source='yahoo' , start = start ,end= end )
    return data

dfnew = getMyPortfolioNew(stocksymbols)

In [175]:
dfnew.ta.strategy(CustomStrategy)

In [176]:
dfnew.rename(columns = {'EMA_5':'Short', 'EMA_10':'Middle', 'EMA_20':'Long'}, inplace = True)
dfnew['OBV_EMA'] = dfnew['OBV'].ewm(span=14).mean()


In [177]:
dfnew['buysignal'] = buy_sell_function(dfnew)[0]
dfnew['buyprice'] =buy_sell_function(dfnew)[1]

obv1=dfnew['OBV']
obvema1 = dfnew['OBV_EMA']
fl=0
obvsig1=[]
for i in range(len(obv)):
  if obv1[i]<obvema1[i] and fl==1:
    obvsig1.append(-1)
    fl=0
  elif obv1[i]>obvema1[i] and fl==0:
    obvsig1.append(1)
    fl=1
  else:
    obvsig1.append(0)

stoch1=dfnew['STOCHRSIk_14_14_3_3']
stochsig1=[]
for i in range(len(stoch1)):
  if stoch1[i]>80:
    stochsig1.append(-1)
  elif stoch1[i]<20:
    stochsig1.append(1)
  else:
    stochsig1.append(0)

mfi1=dfnew['MFI_14']
rsi1=dfnew['RSI_14']
mfisig1=[]
for i in range(len(mfi)):
  if mfi1[i]>80:
    mfisig1.append(-1)
  elif mfi1[i]<20:
    mfisig1.append(1)
  else:
    mfisig1.append(0)

rsisig1=[]
for i in range(len(rsi1)):
  if rsi1[i]>70:
    rsisig1.append(-1)
  elif rsi1[i]<30:
    rsisig1.append(1)
  else:
    rsisig1.append(0)

bs1=dfnew['buysignal']
ss1=dfnew['buyprice']

In [178]:

def calcProfit1(w1,w2,w3,w4,w5):
  tp=0
  fl=0
  bp=0
  z=0
  buypricefirst=1
  for i in range(len(bs)):
    val= mfisig1[i]*w1 + obvsig1[i]*w2 + rsisig1[i]*w3 + stochsig1[i]*w4 + bs1[i]*w5 
    if val>=20 and fl==0:
      if z==0:
        buypricefirst=ss1[i]
      
      tp-=ss1[i]
      z=z+1
      fl=1
      bp=ss1[i]
    elif val<=-20 and fl==1:
      tp+=ss1[i]
      fl=0

  if(fl==1):
    tp+=bp

  return tp/buypricefirst

funcval=calcProfit1(wt[0],wt[1],wt[2],wt[3],wt[4])
print(wt)

fin1=(funcval)*100
marketprof1=((ss1[len(ss)-1]-ss1[0])/ss1[0])*100
print(fin1)
print(marketprof1)

[80, 80, 80, 80, 10]
1.4381874302687492
-5.046632910770824
